In [2]:
import sys; sys.path.insert(0,'../../..')

from lltk.imports import *
from corpus_graph import *
from lltk.corpus.wikidata import *
# setup_log(to_screen=True)

In [3]:
t=Text(author='Joyce', title='Portrait of the Artist', y=123231, corpus='tmpcorp')
# x = t.metadata()
# t.wikidata.au

[14:30:35] lltk.corpus.corpus.Corpus():1405: Corpus begun: [BaseCorpus](tmpcorp)


In [5]:
def get_qid(*x,**y): return get_wikidata_id(*x,**y)

def get_wikidata_id(
        text=None,
        meta={},
        idkey='_id_wikidata',
        addrkey='_addr_wikidata',
        prefix='_wikidata/',
        corpus_id='wikidata',
    ):
    tmeta = text.metadata(from_sources=True) if not meta and text is not None else meta
    log.debug(f'Tmeta = {tmeta}')
    Qid=''
    Qaddr = tmeta.get(addrkey,'').split('/')[-1]
    Qid = tmeta.get(idkey,Qid)
    if not Qid: Qid = Qaddr
    if not Qid: Qid = get_wikidata_id(text)
    Qid=Qid.replace('(','').replace(')','')
    return Qid

def query_wikidata_id(
        qstr_or_text,
        lang="en",
        what=["work","manuscript","text"],
        min_match_ratio_au=50,
        min_match_ratio_ti=50,
        timeout=30,
        verbose=True,
        **kwargs):
    import requests,wptools,bs4
    from fuzzywuzzy import fuzz

    if issubclass(type(qstr_or_text), BaseText):
        text = qstr_or_text
        qstr=wikidata_query_str(qstr_or_text)
    else:
        text = None
        qstr=qstr_or_text

    if verbose: log.debug('Querying wikidata: '+qstr)
    
    safe='+'.join(clean_text(qstr).split())
    url=f'https://www.wikidata.org/w/index.php?search={safe}'
    with requests.Session() as s:
        html = s.get(url,timeout=timeout).text
    dom=bs4.BeautifulSoup(html,'html')

    for item in dom.select('.wb-itemlink-id'):
        itext=item.text
        qid=itext.replace('(','').replace(')','')
        print(qid)
    return ''


In [6]:
get_qid(t)

[14:30:42] __main__.get_wikidata_id():12: Tmeta = {'id': 'Joyce.Portrait_of_the_Artist', '_addr': '_tmpcorp/Joyce.Portrait_of_the_Artist', 'author': 'Joyce', 'corpus': 'wikidata', 'title': 'Portrait of the Artist', 'y': 123231}
[14:30:42] __main__.get_wikidata_id():12: Tmeta = {'id': 'Joyce.Portrait_of_the_Artist', '_addr': '_tmpcorp/Joyce.Portrait_of_the_Artist', 'author': 'Joyce', 'corpus': 'wikidata', 'title': 'Portrait of the Artist', 'y': 123231}
[14:30:42] __main__.get_wikidata_id():12: Tmeta = {'id': 'Joyce.Portrait_of_the_Artist', '_addr': '_tmpcorp/Joyce.Portrait_of_the_Artist', 'author': 'Joyce', 'corpus': 'wikidata', 'title': 'Portrait of the Artist', 'y': 123231}
[14:30:42] __main__.get_wikidata_id():12: Tmeta = {'id': 'Joyce.Portrait_of_the_Artist', '_addr': '_tmpcorp/Joyce.Portrait_of_the_Artist', 'author': 'Joyce', 'corpus': 'wikidata', 'title': 'Portrait of the Artist', 'y': 123231}
[14:30:42] __main__.get_wikidata_id():12: Tmeta = {'id': 'Joyce.Portrait_of_the_Artist',

In [9]:
def get_wikidata(
        qid,
        text=None,
        verbose=False,
        lang="en",
        what={"work","manuscript","text"},
        min_match_ratio_au=50,
        min_match_ratio_ti=50,
        qstr='',
        timeout=30):
    #qid=qid.split('/')[-1]
    
    import wptools
    page = wptools.page(wikibase=qid, silent=not verbose)
    o = page.get_wikidata()
    if os is None or not hasattr(o,'data'): return
    data=o.data
    whatres = data.get('what','')
    for whatx in what:
        if not whatres or not whatx or whatx in whatres:
            odat=data.get('wikidata')
            od = format_wikidata_d(odat)
            wd_au,wd_ti = wikidata_get_author(od), wikidata_get_title(od)
            if not wd_ti: wd_ti=data.get('label')
            # pprint([wd_au,wd_ti,od])
            if not wd_au or not wd_ti: continue

            if text is not None:
                t_au,t_ti = text.au, text.shorttitle
                au_ratio=fuzz.token_set_ratio(wd_au, t_au)
                ti_ratio=fuzz.token_set_ratio(wd_ti, t_ti)
                if min_match_ratio_au and au_ratio < min_match_ratio_au: continue
                if min_match_ratio_ti and ti_ratio < min_match_ratio_ti: continue
            else:
                ti_ratio=np.nan
                au_ratio=np.nan

            return dict(
                qid=qid,
                meta=od,
                meta_simple=format_wikidata_d_simple(od),
                wd_title=wd_ti,
                wd_author=wd_au,
                wd_author_match=au_ratio,
                wd_title_match=ti_ratio,
                query=qstr,
            )
    return {}

In [10]:
get_wikidata(qid)

{}

In [5]:
t.wikidata

[14:23:20] lltk.imports.plog():67: 
           id:
           Q4658974-->Joyce.Portrait_of_the_Artist
[14:23:20] lltk.imports.plog():67: 
           Cached: /Users/ryan/lltk_data/corpora/tmpcorp/texts/Joyce.Portrait_of_the_Artist/meta.json
           ({'_addr': '_tmpcorp/Joyce.Portrait_of_the_Artist', 'author': 'Joyce', 'corpus': 'wikidata', 'id': 'Joyce.Portrait_of_the_Artist', 'title': 'Portrait of the Artist', 'y': 123231},)
[14:23:20] lltk.corpus.wikidata.wikidata.get_qid():40: Tmeta = {'id': 'Joyce.Portrait_of_the_Artist', '_addr': '_tmpcorp/Joyce.Portrait_of_the_Artist', 'author': 'Joyce', 'corpus': 'wikidata', 'title': 'Portrait of the Artist', 'y': 123231}
[14:23:20] lltk.corpus.wikidata.wikidata.get_wikidata_id():336: Querying wikidata: Joyce, Portrait of the Artist
[14:23:23] lltk.imports.plog():67: 
           [{'id': 'Joyce.Portrait_of_the_Artist', '_addr': '_tmpcorp/Joyce.Portrait_of_the_Artist', 'author': 'Joyce', 'corpus': 'wikidata', 'title': 'Portrait of the Artist', '

[TextWikidata](_wikidata/Q4658974)

In [6]:
get_qid(t)

[14:06:32] lltk.text.text.cache_meta_json():142: Cache unchanged
[14:06:32] lltk.text.text.cache_meta_json():142: Cache unchanged


NameError: name 'qstr' is not defined

In [3]:
# get_qid(t)

In [4]:
init_wikidata(t)

[14:06:16] lltk.imports.plog():67: 
           text = [BaseText](_Test2/Joyce.Portrait_of_the_Artist)()
[14:06:16] lltk.text.text.cache_meta_json():142: Cache unchanged
[14:06:16] lltk.corpus.corpus.Corpus():1401: Corpus loaded: [Wikidata](wikidata)
[14:06:16] lltk.text.text.cache_meta_json():142: Cache unchanged


NameError: name 'qstr' is not defined

In [5]:
t.init_meta_json()

{'id': 'Joyce.Portrait_of_the_Artist',
 '_addr': '_Test2/Joyce.Portrait_of_the_Artist',
 'author': 'Joyce',
 'title': 'Portrait of the Artist',
 'y': 123231,
 '_addr_wikidata': '_wikidata/Q465360'}

'Joyce, Portrait of the Artist'

In [6]:
t.wikidata

[TextWikidata](_wikidata/Q465360)

In [7]:
t.wikidata.meta

[12:49:11] lltk.text.text.cache_meta_json():142: Cache unchanged


TypeError: lltk.text.text.BaseText.metadata() got multiple values for keyword argument 'from_json'

In [10]:
sources(tw)

NameError: name 'tw' is not defined

In [5]:
t.sources

[]


[]

In [6]:
tw=t.wikidata
tw.sources

[19:30:31] lltk.corpus.wikidata.wikidata.wikidata_search():236: Querying wikidata: Joyce, Portrait of the Artist
[19:30:36] lltk.corpus.corpus.Corpus():1398: Corpus loaded: [Wikidata](wikidata)
[19:30:36] lltk.corpus.corpus.init_meta_files():281: Initializing from files: [Wikidata](wikidata)
[19:30:36] lltk.text.text.cache_meta_json():135: Cached: /Users/ryan/lltk_data/corpora/wikidata/texts/Q465360/meta.json
[19:30:36] lltk.text.text.cache_meta_json():135: Cached: /Users/ryan/lltk_data/corpora/wikidata/texts/Q465360/meta.json
[19:30:36] lltk.text.text.cache_meta_json():135: Cached: /Users/ryan/lltk_data/corpora/tmp_corpus/texts/Joyce.Portrait_of_the_Artist/meta.json
[19:30:36] lltk.text.text.cache_meta_json():135: Cached: /Users/ryan/lltk_data/corpora/wikidata/texts/Q465360/meta.json
[19:30:36] lltk.text.text.cache_meta_json():135: Cached: /Users/ryan/lltk_data/corpora/wikidata/texts/Q465360/meta.json
[19:30:36] lltk.text.text.cache_meta_json():135: Cached: /Users/ryan/lltk_data/corpo

In [9]:
tw.sources

[19:30:45] lltk.text.text.cache_meta_json():135: Cached: /Users/ryan/lltk_data/corpora/wikidata/texts/Q465360/meta.json
[19:30:45] lltk.text.text.cache_meta_json():135: Cached: /Users/ryan/lltk_data/corpora/wikidata/texts/Q465360/meta.json


In [11]:
x=tw.sources

{'_addr': '_wikidata/Q465360',
 'id': 'Q465360',
 '_is': ['_tmp_corpus/Joyce.Portrait_of_the_Artist'],
 '_addr_tmp_corpus': '_tmp_corpus/Joyce.Portrait_of_the_Artist'}

[19:26:24] lltk.text.text.cache_meta_json():135: Cached: /Users/ryan/lltk_data/corpora/wikidata/texts/Q465360/meta.json


{'_addr': '_wikidata/Q465360',
 'id': 'Q465360',
 '_is': ['_tmp_corpus/Joyce.Portrait_of_the_Artist'],
 '_addr_tmp_corpus': '_tmp_corpus/Joyce.Portrait_of_the_Artist'}

[19:26:24] lltk.text.text.cache_meta_json():135: Cached: /Users/ryan/lltk_data/corpora/wikidata/texts/Q465360/meta.json


In [13]:
[x]

[None]

In [5]:
# !cat /Users/ryan/lltk_data/corpora/wikidata/texts/Q465360/meta.json

{
    "_addr": "_wikidata\/Q465360",
    "query": "Joyce, A Portrait of the Artist as a Young Man",
    "instance_of|P31": "literary work|Q7725634",
    "author|P50": "James Joyce|Q6882",
    "publisher|P123": [
        "The Egoist|Q2919056",
        "B. W. Huebsch|Q4834254"
    ],
    "follows|P155": [
        "Dubliners|Q459592",
        "Stephen Hero|Q2070762"
    ],
    "genre|P136": [
        "autobiographical novel|Q3056541",
        "K\u00fcnstlerroman|Q783459",
        "Bildungsroman|Q223945"
    ],
    "isbn-10|P957": "0-14-243734-4",
    "freebase_id|P646": "\/m\/01vj5q",
    "bibliotheque_nationale_de_france_id|P268": "119503132",
    "followed_by|P156": "Ulysses|Q6511",
    "movement|P135": "modernist literature|Q971480",
    "project_gutenberg_ebook_id|P2034": "4217",
    "language_of_work_or_name|P407": "English|Q1860",
    "narrative_location|P840": "Dublin|Q1761",
    "publication_date|P577": "+1916-00-00T00:00:00Z",
    "country_of_origin|P495": "Republic of Ireland|Q2

In [10]:
tw.addr

'_wikidata/Joyce.Portrait_of_the_Artist'

In [6]:
t.metadata(force=True)

{'_addr': '_tmp_corpus/Joyce.Portrait_of_the_Artist',
 'author': 'Joyce',
 'title': 'Portrait of the Artist',
 'id': 'Joyce.Portrait_of_the_Artist'}

In [4]:
t.wikidata.meta

{'query': 'Joyce, A Portrait of the Artist as a Young Man',
 'instance_of|P31': 'literary work|Q7725634',
 'author|P50': 'James Joyce|Q6882',
 'publisher|P123': ['The Egoist|Q2919056', 'B. W. Huebsch|Q4834254'],
 'follows|P155': ['Dubliners|Q459592', 'Stephen Hero|Q2070762'],
 'genre|P136': ['autobiographical novel|Q3056541',
  'Künstlerroman|Q783459',
  'Bildungsroman|Q223945'],
 'isbn-10|P957': '0-14-243734-4',
 'freebase_id|P646': '/m/01vj5q',
 'bibliotheque_nationale_de_france_id|P268': '119503132',
 'followed_by|P156': 'Ulysses|Q6511',
 'movement|P135': 'modernist literature|Q971480',
 'project_gutenberg_ebook_id|P2034': '4217',
 'language_of_work_or_name|P407': 'English|Q1860',
 'narrative_location|P840': 'Dublin|Q1761',
 'publication_date|P577': '+1916-00-00T00:00:00Z',
 'country_of_origin|P495': 'Republic of Ireland|Q27',
 'characters|P674': ['Stephen Dedalus|Q543671',
  'Simon Dedalus|Q7518595',
  'May Goulding Dedalus|Q16910626',
  'Dante Riordan|Q28657003',
  'Charles Stewar

In [8]:
from lltk.corpus.wikidata import *
# query_wikidata(t)

In [13]:
t.metadata(from_wikidata=True)

{'_addr': '_tmp_corpus/X421756',
 'query': 'Joyce, A Portrait of the Artist as a Young Man',
 'instance_of|P31': 'literary work|Q7725634',
 'author|P50': 'James Joyce|Q6882',
 'publisher|P123': ['The Egoist|Q2919056', 'B. W. Huebsch|Q4834254'],
 'follows|P155': ['Dubliners|Q459592', 'Stephen Hero|Q2070762'],
 'genre|P136': ['autobiographical novel|Q3056541',
  'Künstlerroman|Q783459',
  'Bildungsroman|Q223945'],
 'isbn-10|P957': '0-14-243734-4',
 'freebase_id|P646': '/m/01vj5q',
 'bibliotheque_nationale_de_france_id|P268': '119503132',
 'followed_by|P156': 'Ulysses|Q6511',
 'movement|P135': 'modernist literature|Q971480',
 'project_gutenberg_ebook_id|P2034': '4217',
 'language_of_work_or_name|P407': 'English|Q1860',
 'narrative_location|P840': 'Dublin|Q1761',
 'publication_date|P577': '+1916-00-00T00:00:00Z',
 'country_of_origin|P495': 'Republic of Ireland|Q27',
 'characters|P674': ['Stephen Dedalus|Q543671',
  'Simon Dedalus|Q7518595',
  'May Goulding Dedalus|Q16910626',
  'Dante Rior

In [4]:
t.wikidata

AttributeError: 'NoneType' object has no attribute 'id'

In [10]:
CG = CorpusGraph('markmark,txtlab')
g = CG.graph(only_identical=True)

[MarkMark] Iterating texts:   0%|          | 0/355 [00:00<?, ?it/s]

[TxtLab] Iterating texts:   0%|          | 0/450 [00:00<?, ?it/s]

[CorpusGraph] Iterating texts:   0%|          | 0/805 [00:00<?, ?it/s]

In [14]:
g.order(), g.size()

(1899, 1094)

In [15]:
text_graph = CG.graph_identity_texts()
text_graph.order(), text_graph.size()

[CorpusGraph] Iterating texts:   0%|          | 0/805 [00:00<?, ?it/s]

(1094, 289)

In [30]:
edges = list(text_graph.edges(data=True))

In [31]:
Text(author=)

In [29]:
# u,v,d = random.choice(edges)
# uT=Text(u)
# vT=Text(v)
# u,v,d,uT,vT,uT._is,vT._is

In [13]:
for nodegrp in CG.iter_identity_texts(): print(nodegrp)

[CorpusGraph] Iterating texts:   0%|          | 0/805 [00:00<?, ?it/s]

  0%|          | 0/1094 [00:00<?, ?it/s]

[TextMarkMark](_markmark/Ellis_Bret_Easton.American_Psycho)
[TextMarkMark](_markmark/Beerbohm_Max.Zuleika_Dobson)
[TextWikidata](_wikidata/Q646361)
[TextWikidata](_wikidata/Q264451)
[TextWikidata](_wikidata/Q1093807)
[TextMarkMark](_markmark/Greene_Graham.The_Heart_of_the_Matter)
[TextTxtLab](_txtlab/DE_1789_SchillerFriedrich_DerGeisterseher_Novel)
[TextTxtLab](_txtlab/DE_1791_NaubertBenedikte_AlfvonDulmen_Novel)
[TextTxtLab](_txtlab/DE_1866_MarlittEugenie_Goldelse_Novel)
[TextTxtLab](_txtlab/EN_1920_FitzgeraldFScott_ThisSideofParadise_Novel)
[TextTxtLab](_txtlab/FR_1821_LatoucheHenriDe_DernieresLettresDeDeuxAmansDeBarcelone_Novel)
[TextTxtLab](_txtlab/EN_1838_MartineauHarriet_Deerbrook_Novel)
[TextTxtLab](_txtlab/FR_1908_LerouxGaston_LeParfumdelaDameenNoir_Novel)
[TextMarkMark](_markmark/Larsson_Stieg.The_Girl_Who_Kicked_the_Hornets_Nest)
[TextMarkMark](_markmark/Coetzee_J.M..Waiting_for_the_Barbarians)
[TextMarkMark](_markmark/Kazan_Elia.The_Arrangement)
[TextMarkMark](_markmark/Wrig

In [26]:
goethe_sorrows = Text('DE_1774_Goethe,Johann_DieLeidendesjungenWerthers1_Novel', 'txtlab')
goethe_sorrows.meta

{'_addr': '_txtlab/DE_1774_GoetheJohann_DieLeidendesjungenWerthers1_Novel',
 'id': 'DE_1774_Goethe,Johann_DieLeidendesjungenWerthers1_Novel'}

In [18]:
g = graph_identity_texts(CG)
list(g.neighbors('_wikidata/Q213019'))

[CorpusGraph] Iterating texts:   0%|          | 0/2476 [00:00<?, ?it/s]

['_markmark/Wells_H._G..The_War_of_the_Worlds']

In [7]:
Text('_wikidata/Q213019').meta

{'_addr': '_wikidata/Q213019',
 'author': 'H.G. Wells',
 'award_received': ['NPR_Top_100_Science_Fiction_and_Fantasy_Books',
  "Le_Monde's_100_Books_of_the_Century"],
 'biblioteca_nacional_de_espana_id': 'XX2230813',
 'commons_category': 'The War of the Worlds (1898)',
 'copyright_status': ['public_domain', 'public_domain'],
 'country_of_origin': 'United Kingdom',
 'derivative_work': ['The_War_of_the_Worlds',
  'War_of_the_Worlds',
  'War_of_the_Worlds',
  'The_War_of_the_Worlds',
  'War_of_the_Worlds_2:_The_Next_Wave',
  "H._G._Wells'_War_of_the_Worlds",
  'The_War_of_the_Worlds',
  'War_of_the_Worlds'],
 'encyclopaedia_britannica_online_id': 'topic/The-War-of-the-Worlds-novel-by-Wells',
 'fantlab_work_id': '9457',
 'first_line': 'No one would have believed in the last years of the nineteenth century that this world was being watched keenly and closely by intelligences greater than man’s and yet as mortal as his own; that as men busied themselves about their various concerns they were

In [12]:
# for node in iter_identity_texts(CG):
#     if len(node._is):
#         print(node)
#         for n in node._is:
#             print(f' ...is also: {n}')
#         print()


In [28]:
t=Text('_markmark/Faulkner_William.The_Sound_and_the_Fury')
t.metadata(force=True)

{'_addr': '_markmark/Faulkner_William.The_Sound_and_the_Fury',
 'author': 'Faulkner, William',
 'corpus': 'MarkMark',
 'dob': 1897,
 'dod': 1962,
 'gender': 'M',
 'name_first': 'William',
 'name_last': 'Faulkner',
 'nation': 'American',
 'num_words': 112789,
 'ocr_accuracy': 0,
 'title': 'The Sound and the Fury',
 'year': 1929,
 '_addr_wikidata': '_wikidata/Q135515',
 'from_text_func': 1,
 'name_middle': 'Cuthbert',
 'id': 'Faulkner_William.The_Sound_and_the_Fury',
 '_id_wikidata': 'Q135515'}

In [25]:
t

[TextWikidata](_wikidata/Q135515)

In [7]:
g.add_edge(1,2,x=10,y=20)

0

In [10]:
g.get_edge_data(1,2)

{0: {'x': 10, 'y': 20}}